<h1><center>Laboratorio 6: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Camilo Carvajal Reyes
- Nombre de alumno 2: Andrés Gonzalez Fuentes


### **Link de repositorio de GitHub:** `https://github.com/camilocarvajalreyes/laboratorios-MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- Fecha de entrega: 09/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la farbicación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [2]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df_retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400947 entries, 0 to 400946
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      400947 non-null  category      
 1   StockCode    400947 non-null  category      
 2   Description  400947 non-null  object        
 3   Quantity     400947 non-null  int64         
 4   InvoiceDate  400947 non-null  datetime64[ns]
 5   Price        400947 non-null  float64       
 6   Customer ID  400947 non-null  category      
 7   Country      400947 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 15.9+ MB


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.
- **Recency (R)**: Indica la actualidad de la interacción de un cliente con la empresa, y da información sobre la tendencia a repetir la compra. Se define como: 
$$Recency(n)=\dfrac{1}{n} \sum^n_{i=1} date\_diff(t_{fecha final}, t_{m-i+1})$$

 Donde $date\_diff$ representa la diferencia en días entre la fecha de finalización del periodo de observación ($t_{fecha final}$), y la fecha de una visita del cliente cercana a $t_{fecha final}$, $t_{m-i+1}; t_{m}$ es la última visita del cliente; y n es el número de visitas recientes del cliente consideradas.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**Nota:** Para la $fechafinal$ utilice la fecha máxima del dataset más 1 día.

**Ejemplo de Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [4]:
def series(customer,column,df_in=df_retail):
    """Función auxiliar que retorna una serie asociada a las ocurrencias de un clienete particular"""
    return df_in[df_in['Customer ID'] == customer][column]

def sub_df(customer,df_in=df_retail):
    """Función auxiliar que retorna una dataframe asociado a las ocurrencias de un clienete particular"""
    return df_in[df_in['Customer ID'] == customer]

def custom_features(dataframe_in):
    df = dataframe_in[['Customer ID']].copy(deep=True)
    df = df.drop_duplicates()
    # variables útiles
    t_final = df_retail['InvoiceDate'].max() + pd.DateOffset(1)
    # LRMPF
    delta_fecha = lambda row: (series(row['Customer ID'],'InvoiceDate').max() - series(row['Customer ID'],'InvoiceDate').min()).days
    df['Length'] = df.apply(delta_fecha,axis=1)

    recency = lambda row: ((series(row['Customer ID'],'InvoiceDate').drop_duplicates()).apply(lambda date: (t_final-date).days)).mean()
    df['Recency'] = df.apply(lambda val: round(recency(val)),axis=1)

    monetary = lambda row: sub_df(row['Customer ID']).groupby(['InvoiceDate']).Price.sum().mean()
    df['Monetary'] = df.apply(monetary,axis=1)

    frequency = lambda row: len(series(row['Customer ID'],'InvoiceDate').drop_duplicates())
    df['Frequency'] = df.apply(frequency,axis=1)

    periodicity = lambda row: ((series(row['Customer ID'],'InvoiceDate').drop_duplicates()).diff().apply(lambda val: val if pd.isnull(val) else val.days)).std()
    df['Periodicity'] = df.apply(lambda val: 0 if pd.isnull(periodicity(val)) else round(periodicity(val)),axis=1)

    return df

In [5]:
# df_LRMPF = custom_features(df_retail[df_retail['Customer ID'].isin([12346.0,12347.0,12349.0,12352.0,12356.0])])
df_LRMPF = custom_features(df_retail)
# print('Presenta un total de {} filas con Customer IDs duplicados.'.format(df_LRMPF.duplicated('Customer ID').sum()))

In [6]:
df_LRMPF[df_LRMPF['Customer ID'].isin([12346.0,12347.0,12349.0,12352.0,12356.0])].sort_values('Customer ID')

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
20724,12346.0,196,325,18.760000,11,37
326853,12347.0,37,22,81.475000,2,0
131582,12349.0,181,158,291.780000,3,102
349604,12352.0,16,20,27.300000,2,0
293951,12356.0,44,35,86.476667,3,13


**debugging**

In [7]:
df_retail[df_retail['Customer ID']==12352.0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
349604,532430,85152,HAND OVER THE CHOCOLATE SIGN,12,2010-11-12 10:20:00,2.10,12352.0,Norway
349605,532430,21166,COOK WITH WINE METAL SIGN,12,2010-11-12 10:20:00,1.95,12352.0,Norway
349606,532430,22668,PINK BABY BUNTING,5,2010-11-12 10:20:00,2.95,12352.0,Norway
349607,532430,21121,SET/10 RED POLKADOT PARTY CANDLES,24,2010-11-12 10:20:00,1.25,12352.0,Norway
349608,532430,82583,HOT BATHS METAL SIGN,12,2010-11-12 10:20:00,2.10,12352.0,Norway
349609,532430,21175,GIN + TONIC DIET METAL SIGN,12,2010-11-12 10:20:00,2.10,12352.0,Norway
382345,535838,84970L,SINGLE HEART ZINC T-LIGHT HOLDER,12,2010-11-29 10:07:00,0.95,12352.0,Norway
382346,535838,22423,REGENCY CAKESTAND 3 TIER,1,2010-11-29 10:07:00,12.75,12352.0,Norway
382347,535838,22198,LARGE POPCORN HOLDER,12,2010-11-29 10:07:00,1.65,12352.0,Norway
382348,535838,22197,SMALL POPCORN HOLDER,12,2010-11-29 10:07:00,0.85,12352.0,Norway


In [8]:
df_retail[df_retail['Customer ID']==12352.0].groupby(['InvoiceDate']).Price.sum()

InvoiceDate
2010-11-12 10:20:00    12.45
2010-11-29 10:07:00    42.15
Name: Price, dtype: float64

In [9]:
df_retail['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

**Resúesta de Pauta:**

## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [11]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        Xmin, Xmax = X.min(), X.max()
        return X.apply(lambda val: (val - Xmin)/(Xmax - Xmin))

### 1.3.2 `T-SNA` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo T-sne de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [17]:
minmax_transformer = ColumnTransformer(
    transformers=[
        ('MinMaxScaler',MinMax(), ['Length','Recency','Frequency','Monetary','Periodicity'])
])

lrmfp_tsne_pipeline = Pipeline([
    # aplicar transformaciones para custom features
    ('LRMFP',FunctionTransformer(custom_features)),
    ('MinMaxScaler',minmax_transformer),
    ('TSNE',TSNE())
])

TypeError: 'ColumnTransformer' object is not callable

In [16]:
df_LRMPF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4314 entries, 0 to 400593
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Customer ID  4314 non-null   category
 1   Length       4314 non-null   int64   
 2   Recency      4314 non-null   int64   
 3   Monetary     4314 non-null   float64 
 4   Frequency    4314 non-null   int64   
 5   Periodicity  4314 non-null   int64   
dtypes: category(1), float64(1), int64(4)
memory usage: 373.4 KB


In [15]:
X_tsne = lrmfp_tsne_pipeline.fit_transform(df_retail)

/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


ValueError: Input X contains NaN.
TSNE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
fig = px.scatter(
    df_LRMPF, x=X_tsne[0] y=X_tsne[1], title='Visualización de atributos LRMPF con t-SNE'
)

fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero permutando el algoritmo de reducción de dimensionalidad por k-means.

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab** solo para tareas. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>